# Center Loss in Keras

### Item recognition 'the sequel' - using Center Loss with Transfer Learning

Using InceptionV3

The implementation of **Center Loss** came from: [handongfeng/MNIST-center-loss](https://github.com/handongfeng/MNIST-center-loss/blob/master/centerLoss_MNIST.py)

**Articles:**
- [A Discriminative Feature Learning Approach for Deep Face Recognition](https://ydwen.github.io/papers/WenECCV16.pdf)
- [Understanding Center Loss Based Network for Image Retrieval with Few Training Data](https://openaccess.thecvf.com/content_ECCVW_2018/papers/11132/Ghosh_Understanding_Center_Loss_Based_Network_for_Image_Retrieval_with_Few_ECCVW_2018_paper.pdf)

In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
%matplotlib notebook
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.keras.preprocessing import image_dataset_from_directory # cant import in Tensorflow v2.2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.regularizers import l2
from tensorflow.keras import losses
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt
import numpy as np
import random
from numpy.random import default_rng

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


In [3]:
#import os

In [4]:
from centerLoss import prelu, zero_loss, my_model
from centerLoss import CenterLossLayer

In [5]:
from utils import tbProjector, PCAPlotter, create_testdata

In [6]:
#tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Counting Training Data 

In [8]:
image_dir = "image_data"
train_dir = os.path.join(image_dir, "train")
print(train_dir)

train_classnum = len(os.listdir(train_dir))
print("items = {}".format(len(os.listdir(train_dir))))

image_data/train
items = 706


In [9]:
all_images = 0
class_list = sorted(os.listdir(train_dir))
for i in class_list:
    #print(i)
    all_images += len(os.listdir(os.path.join(train_dir, i)))
print("All image files = {}".format(all_images))
print("class_list[:10] = {}".format(class_list[:10]))

All image files = 12070
class_list[:10] = ['aubeer2_0', 'aubeer2_1', 'aubeer2_10', 'aubeer2_11', 'aubeer2_12', 'aubeer2_14', 'aubeer2_15', 'aubeer2_16', 'aubeer2_17', 'aubeer2_18']


In [10]:
#_BATCH_SIZE = 1024
#_BATCH_SIZE = 256
_BATCH_SIZE = 64

## Setup train data generator

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'image_data/train',
    target_size=(224, 224),
    batch_size=_BATCH_SIZE,
    class_mode='categorical'
    ) 

Found 12070 images belonging to 706 classes.


In [12]:
def plot_dataset(image_samples):

    #plt.figure(figsize=(5, 2))
    fig, axs = plt.subplots(len(image_samples)//5, 5)
                            
    count = 0
    for i in range(len(image_samples)//5):
        for j in range(5):
            #print("i = {}, j = {}".format(i, j))
            #plt.subplot(3, 3, 1 + i + j)
            axs[i, j].imshow(image_samples[count])
            plt.xticks([])
            plt.yticks([])
            count+=1
        #plt.title(examples[1][i])
    plt.show()

In [13]:
#X, Y = train_ds
(X, Y) = next(train_generator)
print(X.shape)
print(Y.shape)

plot_dataset(X[:10])
#print(Y[:10])

(64, 224, 224, 3)
(64, 706)


<IPython.core.display.Javascript object>

In [14]:
labelnames = list(train_generator.class_indices.keys())

for y_onehot in Y[:10]:
    idx = np.argmax(y_onehot, axis=0)
    print(labelnames[idx]) 

audrink_68
aubeer2_93
ausnack_5
drink30
aubeer_25
audrink_119
ausnack_105
snack92
austeafood_000065
aubeer2_64


## Load a pre-trained model and attach some fully-connected layers

In [15]:
emb_size = 128

img_input = keras.Input(shape=(224, 224, 3))
train_labels = keras.Input(shape=(train_classnum,))

final_output, side_output = my_model(img_input, train_labels, emb_size)
model = keras.Model(inputs=[img_input, train_labels], outputs=[final_output, side_output])
model.summary()

706


ValueError: too many values to unpack (expected 2)

## compile

In [ ]:
lambda_centerloss = 0.0007
initial_learning_rate = 0.001

optim = optimizers.SGD(lr=initial_learning_rate, momentum=0.9)
model.compile(optimizer=optim,
                  loss=[losses.categorical_crossentropy, zero_loss],
                  #loss_weights=[1, lambda_centerloss])
                  loss_weights=[1, lambda_centerloss])

## Define data generation wrapper

In [ ]:
def datagen_wrapper(gen, batch_size=64):
    dummy = np.zeros((batch_size, 1))
    #dummy2 = np.zeros((x_test.shape[0], 1))
    while True:
        (X, Y) = next(gen)    
        yield([X, Y], [Y, dummy])


## Prepare Tensorboard log

In [ ]:
from datetime import date, time, datetime
import time

epoch_time = int(time.time())

In [ ]:
log_dir = os.path.abspath("logs/train_log")

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
_TB_METADATA = 'metadata.tsv'

print(log_dir)

In [ ]:
# Test Data for Tensorboard & PCA plot
x_tb, x_tb_label_list = create_testdata(datadir='train', tblog_dir=log_dir, metadatafile=_TB_METADATA)
y_tb = np.zeros((x_tb.shape[0], emb_size))

x_tb_label = [ train_generator.class_indices[label] for label in x_tb_label_list ]
print(x_tb_label)

In [ ]:
epochs=10
steps_per_epoch = int(all_images/_BATCH_SIZE)
#steps_per_epoch = 10

reduced_model = keras.Model(inputs=model.input[0], outputs=model.get_layer('side_out').output)
savepath = "result_{}".format(epoch_time)

model.fit(datagen_wrapper(train_generator, _BATCH_SIZE),
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            verbose=1,
            callbacks=[
                    tbProjector(reduced_model, x_tb, y_tb, log_dir, _TB_METADATA),
                    PCAPlotter(plt, model, reduced_model, x_tb, x_tb_label, epochs, lambda_centerloss)])

**NOTE**

- `lambda = 0.01`: Most of the points which represent samples became tightly close to each other, looking like it's just one point in the plot at the end of the first epoch.

- `lambda = 0.0007 - 0.001 - 0.003`: Better. But after around 10th epoch, center loss started to increase AND keep increasing although it was reasonably decreasing until that point. On the other hand, main loss (categorical cross entropy) kept decreasing throughout the training.

**NOTE2:** After I trained this model with epoch 30 and then trained the fine tuned model with epoch 50, the result is bad.
After epoch 10 it starts to overfitting...?

In [ ]:
#plt.savefig('train_plot.jpg')

In [ ]:
#model_filename = "centerloss_" + str(epoch_time) + ".h5"

#model.save(model_filename)
#print("Trained model was saved into {}.".format(model_filename))

In [ ]:
reduced_filename = "reduced_" + str(epoch_time) + ".h5"
reduced_model.save(reduced_filename)
print("Trained model was saved into {}.".format(reduced_filename))

In [ ]:
weights_filename = "weights_" + str(epoch_time) + ".h5"
model.save_weights(weights_filename)
print("Trained model weights were saved into {}.".format(weights_filename))